# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

**DPR**:  Student remarks and answers are marked with the initials **DPR**.
This variable AUGMENTED_DATASET_PENDING controls whether to apply data augmentation, or taking a previously augmented set from disk.

In [ ]:
AUGMENTED_DATASET_PENDING = True

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle
import tensorflow as tf

# TODO: Fill this in based on where you saved the training and testing data
if AUGMENTED_DATASET_PENDING == True:
    training_file = 'train.p'
else:
    training_file = 'train_augmented.p'
    
validation_file= 'valid.p'
testing_file = 'test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
Xo_train, y_train = train['features'], train['labels']
Xo_valid, y_valid = valid['features'], valid['labels']
Xo_test, y_test = test['features'], test['labels']

assert(len(Xo_train) == len(y_train))
assert(len(Xo_valid) == len(y_valid))
assert(len(Xo_test) == len(y_test))

print()
print("Image Shape: {}".format(Xo_train[0].shape))
print()
print("Training Set:   {} samples".format(len(Xo_train)))
print("Validation Set: {} samples".format(len(Xo_valid)))
print("Test Set:       {} samples".format(len(Xo_test)))

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

import numpy as np

# TODO: Number of training examples
n_train = len(Xo_train)

# TODO: Number of testing examples.
n_test = len(Xo_test)

# TODO: What's the shape of an traffic sign image?
image_shape = Xo_train.shape[1:]

# TODO: How many unique classes/labels there are in the dataset.
n_classes = len(np.unique(y_valid))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

print(Xo_train.dtype)


### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

**DPR**: First, let's observe just one sample for each class.

In [ ]:
import matplotlib.pyplot as plt
import random

# Visualizations will be shown in the notebook.
%matplotlib inline

plt.figure(figsize=(15,15))

print("Showing the 1st image found in the X_train set for each class:")

print(n_train,n_classes)
for i_class in range(n_classes):
    for j in range(n_train):
        if y_train[j] == i_class:
            #print(j, "class: ", i_class)
            plt.subplot(10, 10, i_class+1)
            plt.imshow(Xo_train[j])
            break

**DPR:** As we can see in the output above, there is a great variability in the illumination, framing, orientation and size conditions of the samples. This observations seem to be applicable to every class.  Images don't present a good quality nor resolution, which is normal taking into account that they seem obtained from real-life video sequences.

**DPR:** Now, let's observe a list of 50 random samples of any class chosen randomly:

In [ ]:
plt.figure(figsize=(15,15))

print("Showing the 1st 50 images found in the X_train set for a random Class:")

index = random.randint(0,42)
print("Class:", index)
list_class = [j for j in range(n_train) if y_train[j] == index]

list_show = random.sample(list_class, 50)
print(list_show)
i=0
for j in list_show:
    plt.subplot(10,10,i+1)
    plt.imshow(Xo_train[j])
    i +=1


**DPR:** If we execute the above cell several times, we can observe the aforementioned variability in the image conditions for any given class.

**DPR:** Next, we are going to analyse how the samples in the Training, Validation and Test sets are distributed per classes. To do show, let's plot the histogram of each Y set.

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

fig,ax = plt.subplots()
n, bins, patches = ax.hist(y_train, n_classes, histtype='bar')
fig.tight_layout()
plt.show()

fig,ax = plt.subplots()
n, bins, patches = ax.hist(y_valid, n_classes, histtype='bar')
fig.tight_layout()
plt.show()

fig,ax = plt.subplots()
n, bins, patches = ax.hist(y_test, n_classes, histtype='bar')
fig.tight_layout()
plt.show()

**DPR:** A great dispersion is observed in the distribution of signals per classes. In the training set, some classes are represented by more than 2000 images, whilst some other classes are represented by just little more than one hundred. This situation could cause that our neural network is biased to better represented classes, and also that low-represented classes will have a poor classification performance.  To mitigate this, we will use data augmentation later, in a way that lower represented classes are favorized.

**DPR - NOTE**: When starting working with this project, I tried to follow the example shown in the lab by teacher David Silver, when he applied the LeNet-5 model, previously used with MNIST, to the Traffic Signs set. Astonishingly to me, in the videos he showed that LeNet achieved easily a 0.96 accuracy right-away.  However, reproducing the same steps, I could only obtain an accouracy of 0.88 aprox. Then, I noticed that the only difference is that for our project (February cohort) we are already given a Validation set (valid.p file), whereas in the video, David constructs the validation set by splitting the training set.

It was my impression that a possible explanation is that the valid.p we are given now contained a more "difficult" set of samples that if just randomly obtaning the validation set from the training set. I did corroborate this impression by concatenaning the X_train and X_valid sets, and then splitting a 15% of the result for validation... Only doing this, the accuracy increased from 0.88 to 0.95, similar to the results shown in the video.

** In any case, taking into account that we were required to get a minimum validation accuracy of 0.93, this obviously applies to the valid.p file that we are given, not any other set that we could construct. So, I have stuck to the valid.p given. **

I thought that it was interesting to explain my observations here, nevertheless.
Below we can observe how some classes are over/under represented in the validation set, respective to the training set.

For example, we can see that class 40, has relatively 57% more samples in the Validation Set than compared to the Training set. Moreover, this class has few sample images in the training set, which will lead to poor classification performance for that class.

In [ ]:
# I want to analyze further the distribution of classes, in particular de ratio of representation of each class
# in the training set vs. the representation of the class in the validation set.

len_y_val = len(y_valid)
ratio_val_train=[]
n_train_samples_per_class =[]

for i in range(n_classes):
    n_train_samples_per_class.append((y_train == i).sum())
    ratio_class_val = (y_valid == i).sum() / len(y_valid)
    ratio_class_train = (n_train_samples_per_class[i]/ len(y_train))
    ratio_val_train.append(ratio_class_val/ratio_class_train)
    print("class", i, ratio_class_train, "ratio val/train: ", ratio_val_train[i])

print("\nMean, Std of ratio val/train:", np.mean(ratio_val_train), np.std(ratio_val_train))


## Data Augmentation on the Training Set.

**DPR:** I will augment the data by applying random Translation, Rotation and Shear to the training images.
I will augment so that every class has at least 1000 samples. This means that some of the classes will not require augmentation, whilst others will have an important increase of samples.
I will not augment the data in terms of brightness or contrast, I think the samples provide enough variability on that sense.

In [ ]:
import math
import cv2

def transform_image(img, trans_var=5, ang_var=10, shear_var=5):
    '''
    This function transforms the input image based on a random uniform distribution applied to 3 different
     transformation parameters: translation, rotation and shear(perspective shift).
    img:       input image
    trans_var: desired variation range for translation.
    ang_var:   desired variation range for rotation.
    shear_var: desired variation range for shear.

    Returns: transformed image of the same size as the input.

    '''
    # Calculate random angle and Matrix for Rotation.
    ang_rot = np.random.uniform(ang_var)-ang_var/2
    rows,cols,ch = img.shape    
    Rot_M = cv2.getRotationMatrix2D((cols/2,rows/2),ang_rot,1)

    # Calculate random values for x,y translation and Matrix for Translation.
    tr_x = trans_var*np.random.uniform()-trans_var/2
    tr_y = trans_var*np.random.uniform()-trans_var/2
    Trans_M = np.float32([[1,0,tr_x],[0,1,tr_y]])

    # Calculate perspective points and Matrix for Shear.
    pts1 = np.float32([[5,5],[20,5],[5,20]])
    pt1 = 5+shear_var*np.random.uniform()-shear_var/2
    pt2 = 20+shear_var*np.random.uniform()-shear_var/2
    pts2 = np.float32([[pt1,5],[pt2,pt1],[5,pt2]])
    shear_M = cv2.getAffineTransform(pts1,pts2)

    # We apply cv2 warpAffine function with the 3 previously calculating matrices.
    img = cv2.warpAffine(img,Rot_M,(cols,rows))
    img = cv2.warpAffine(img,Trans_M,(cols,rows))
    img = cv2.warpAffine(img,shear_M,(cols,rows))

    return img


MIN_SAMPLES_PER_CLASS = 1000

def augment_training_dataset(X, y, n_classes):
    
    initial_train_images = len(X)
    
    n_train_samples_per_class =[]

    for i in range(n_classes):
        n_train_samples_per_class.append((y == i).sum())

    X_ext_list = []
    y_aug = []
    for i_class in range(n_classes):
        print("processing class", i_class)
        if(n_train_samples_per_class[i_class] < MIN_SAMPLES_PER_CLASS):
            
            image_multiplier = math.ceil(MIN_SAMPLES_PER_CLASS / n_train_samples_per_class[i_class])
            #print("Class ", i_class, "mult: ", image_multiplier)
            
            for j in range(initial_train_images):
                if(y[j] == i_class):
                    #print("looping class:", i_class, "image index:", j)
                    for n_new in range(image_multiplier-1):
                        #print("adding new image number", n_new,"based on image#",j,"to class", i_class)
                        #print(X[j].shape)
                        img = transform_image(X[j])
                        #img = img.reshape(1,32,32,3)
                        X_ext_list.append(img)
                        #Xo_train = np.concatenate((X,img), axis = 0)
                        #print(X.shape)
                        y_aug.append(i_class)
    
    X_ext_np = np.array(X_ext_list)
    y_aug_np = np.array(y_aug)

    Xaug = np.concatenate((X,X_ext_np), axis=0)
    y_aug = np.concatenate((y,y_aug_np), axis=0)
    return Xaug, y_aug


def save_augmented_dataset(X, y):
    #save the X train data set augmented
    train_augmented_file = "./train_augmented.p" 
    output = open(train_augmented_file, 'wb')

    mydict = {'features': 1, 'labels': 2}
    mydict['features'] = X
    mydict['labels'] = y
    pickle.dump(mydict, output)
    output.close()
                        
                        
if AUGMENTED_DATASET_PENDING:
    Xo_train, y_train = augment_training_dataset(Xo_train, y_train, n_classes)
    print("new X:",len(Xo_train))
    print("new y:",len(y_train))
    save_augmented_dataset(Xo_train, y_train)
    print("Augmented Training set saved to disk")
    print("This is the new histogram of the augmented Training Set:")
    
    fig,ax = plt.subplots()
    n, bins, patches = ax.hist(y_train, n_classes, histtype='bar')
    fig.tight_layout()
    plt.show()
    
else:
    print("Augmented Training set already loaded")



**DPR:** Let's illustrate how the Augmentation algorithm would work for a random image:

In [ ]:
import matplotlib.pyplot as plt
import random

# Visualizations will be shown in the notebook.
%matplotlib inline

plt.figure(figsize=(15,15))
index = random.randint(0,34798)
plt.subplot(10, 10, 1)
plt.imshow(Xo_train[index])

for i in range(2,11):
    plt.subplot(10, 10, i)
    img = transform_image(Xo_train[index], trans_var=5, ang_var=10, shear_var=5)
    plt.imshow(img)
    plt.axis('off')

    
print("First image shown below is original, followed by 10 randomly augmented transformations")

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
### Preprocess the data here. Preprocessing steps could include normalization, converting to grayscale, etc.
### Feel free to use as many code cells as needed.

# For now, we are going to implement the LeNet Architecture, without normalization
# Only shuffling the data

from sklearn.utils import shuffle
import cv2

#Xo_train, y_train = shuffle(Xo_train, y_train)


print("Starting normalization...please wait. It takes few seconds")
print(Xo_train.shape)

mRGB2YUV = np.array ( [[0.299, 0.587, 0.114], 
            [-0.14713, -0.28886, 0.436], 
            [0.615, -0.51499, -0.10001]] )

Xyuv_train = np.dot(Xo_train, mRGB2YUV.T)
Xyuv_valid = np.dot(Xo_valid, mRGB2YUV.T)
Xyuv_test  = np.dot(Xo_test, mRGB2YUV.T)

#Xo_train = (.299*Xo_train[:,:,:,0] + .587*Xo_train[:,:,:,1] + .114*Xo_train[:,:,:,2])
#Xo_valid = (.299*Xo_valid[:,:,:,0] + .587*Xo_valid[:,:,:,1] + .114*Xo_valid[:,:,:,2])
#Xo_test = (.299*Xo_test[:,:,:,0] + .587*Xo_test[:,:,:,1] + .114*Xo_test[:,:,:,2])

#Reshape the data for tensorflow
#Xo_train = Xo_train.reshape(Xo_train.shape+(1,))
#Xo_valid = Xo_valid.reshape(Xo_valid.shape+(1,))
#Xo_test = Xo_test.reshape(Xo_test.shape+(1,))



#
#Xo_train = Xo_train.astype('float64')
#Xo_valid = Xo_valid.astype('float64')
#Xo_test = Xo_test.astype('float64')
#
def normalize_image_array(img, nchannels):
    if nchannels == 1:
        for index in range(len(img)):
            img[index,:,:] -= np.mean(img[index,:,:])
            img[index,:,:] /= np.std(img[index,:,:])
    else:
        for index in range(len(img)):
            for channel in range(nchannels):  
                img[index,:,:,channel] -= np.mean(img[index,:,:,channel])
                img[index,:,:,channel] /= np.std(img[index,:,:,channel])
    
                
def normalize_image_array_simple(img):
    norm_img = (img.astype('float64') - 128.0) / 255.0
    return norm_img



Xy_train = Xyuv_train[:,:,:,0]
Xy_valid = Xyuv_valid[:,:,:,0]
Xy_test = Xyuv_test[:,:,:,0]

#print("STD:", np.std(Xy_train[0,:,:]))

for i in range(len(Xy_train)):
    Xy_train[i,:,:]  = cv2.equalizeHist(Xy_train[i,:,:].astype(np.uint8))
    #Xy_train[i,:,:,1]  = cv2.equalizeHist(Xy_train[i,:,:,1].astype(np.uint8))
    #Xy_train[i,:,:,2]  = cv2.equalizeHist(Xy_train[i,:,:,2].astype(np.uint8))
    
for i in range(len(Xy_valid)):
    Xy_valid[i,:,:]  = cv2.equalizeHist(Xy_valid[i,:,:].astype(np.uint8))
    #Xy_valid[i,:,:,1]  = cv2.equalizeHist(Xy_valid[i,:,:,1].astype(np.uint8))
    #Xy_valid[i,:,:,2]  = cv2.equalizeHist(Xy_valid[i,:,:,2].astype(np.uint8))
    
for i in range(len(Xy_test)):
    Xy_test[i,:,:]  = cv2.equalizeHist(Xy_test[i,:,:].astype(np.uint8))
    #Xy_test[i,:,:,1]  = cv2.equalizeHist(Xy_test[i,:,:,1].astype(np.uint8))
    #Xy_test[i,:,:,2]  = cv2.equalizeHist(Xy_test[i,:,:,2].astype(np.uint8))

#print(Xy_train.dtype)
#print("STD:", np.std(Xy_train[0,:,:]))
#print("STD:", np.std(X_train[0,:,:,1]))
#print("STD:", np.std(X_train[0,:,:,2]))

normalize_image_array(Xy_train,1)
normalize_image_array(Xy_valid,1)
normalize_image_array(Xy_test,1)

#print("STD:", np.std(Xy_train[0,:,:]))

print(Xy_train.shape)

X_train = Xy_train
X_valid = Xy_valid
X_test = Xy_test

X_train = X_train.reshape(X_train.shape+(1,))
X_valid = X_valid.reshape(X_valid.shape+(1,))
X_test = X_test.reshape(X_test.shape+(1,))

#print("STD:", np.std(X_train[0,:,:,0]))
#print("STD:", np.std(X_train[0,:,:,1]))
#print("STD:", np.std(X_train[0,:,:,2]))

print(X_train.shape)
print("Normalization finished!.")


X_train, y_train = shuffle(X_train, y_train)

**DPR**: Let's see the results, for the same list of images that we saw at the begining:

In [ ]:
import matplotlib.pyplot as plt
import random

# Visualizations will be shown in the notebook.
%matplotlib inline

index = random.randint(0,len(X_valid))

plt.figure()
plt.imshow(Xo_valid[index])
plt.title("before normalizing")
plt.figure()
plt.imshow(X_valid[index,:,:].reshape(32,32), cmap='gray')
plt.title("after normalizing")

### Model Architecture Hyperparameters

**DPR**: Hyperparameters:  A value between 25 and 30 Epochs seems to work well with my model, reaching almost a 99.9% training accuracy with almost 98% validation accuracy and 95-96% Test accuracy.

Decreasing slightly the default learning rate from 0.001 to 0.00095 seemed to improve the results.

I have tried to apply an exponentially decreasing learning rate, but this seems not to have much effect with the AdamOptimizer, that converges much faster than simple Gradient Descent, and seems to manage the learning rate adapting internally.

In [ ]:
import tensorflow as tf

EPOCHS = 30
BATCH_SIZE = 128

RATE = 0.00095

KEEP_DROPOUT = 0.5

### Model Architecture

**DPR:** My Architecture (MyLeNet) is based basically on the LeNet-5 used in the class lab, introducing the following modifications:
 - Increasing L1 output size from 6 to 12.
 - Increasing L2 output size from 16 to 32.
 - Modified following layers accordingly to the new L1, L2 sizes.
 - Introducing Dropout in training phase, betweeb the FC layers, with a 0.5 probability, that worked better than 0.25 or 0.75.

I also experimented with other enhancements, like adding a new convolutional or fully connected layer, increasing FC layer feature sizes, and even introducing the concept of multi-connected conv layers (concatenating the flattened output of conv1 to the FC0 layer. However, none of these changes improved the results, so I removed them and kept the simple design.

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

from tensorflow.contrib.layers import flatten

def MyLeNet(x, keep_prob):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    L1_OUTPUT_SIZE = 12
    L2_OUTPUT_SIZE = 32
    
    # Layer 1: Convolutional. Input = 32x32x1. Output = 28x28xL1_OUTPUT_SIZE.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 1, L1_OUTPUT_SIZE), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(L1_OUTPUT_SIZE))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b
    
    # Layer 1 Activation.
    #conv1_relu = tf.Variable(tf.zeros(shape=(BATCH_SIZE,28,28,L1_OUTPUT_SIZE)))
    conv1_relu = tf.nn.relu(conv1, name="conv1_relu")

    # Layer 1 Pooling. Input = 28x28x L1_OUTPUT_SIZE. Output = 14x14x L1_OUTPUT_SIZE.
    conv1 = tf.nn.max_pool(conv1_relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    
    
    # Layer 2: Convolutional. Output = 10x10xL2_OUTPUT_SIZE.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, L1_OUTPUT_SIZE, L2_OUTPUT_SIZE), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(L2_OUTPUT_SIZE))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    
    # Layer 2 Activation.
    #conv2_relu = tf.Variable(tf.zeros(shape=(BATCH_SIZE,10,10,L2_OUTPUT_SIZE)))
    conv2_relu = tf.nn.relu(conv2, name="conv2_relu")

    # Layer 2 Pooling. Input = 10x10xL2_OUTPUT_SIZE. Output = 5x5xL2_OUTPUT_SIZE.
    conv2 = tf.nn.max_pool(conv2_relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # Layer 2 Flatten. Input = 5x5xL2_OUTPUT_SIZE. Output = 800.
    fc0   = flatten(conv2)
    
    ######################
    FC1_OUTPUT_SIZE = 120
    FC2_OUTPUT_SIZE = 84
    FC3_OUTPUT_SIZE = 43
    ######################
    
    
    # Layer 3: Fully Connected. Input = 800. Output = FC1_OUTPUT_SIZE.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(800, FC1_OUTPUT_SIZE), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(FC1_OUTPUT_SIZE))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b

    
    # Layer 3 Activation.
    fc1    = tf.nn.relu(fc1)
    
    # Layer 3 Drop-out
    fc1 = tf.nn.dropout(fc1, keep_prob)

    # Layer 4: Fully Connected. Input = FC1_OUTPUT_SIZE. Output = FC2_OUTPUT_SIZE.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(FC1_OUTPUT_SIZE, FC2_OUTPUT_SIZE), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(FC2_OUTPUT_SIZE))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    
    # Layer 4 Activation.
    fc2    = tf.nn.relu(fc2)
    
    # Layer 4 Drop-out
    fc2 = tf.nn.dropout(fc2, keep_prob)

    # Layer 5: Fully Connected. Input = FC2_OUTPUT_SIZE. Output = FC3_OUTPUT_SIZE.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(FC2_OUTPUT_SIZE, FC3_OUTPUT_SIZE), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(FC3_OUTPUT_SIZE))
    logits = tf.matmul(fc2, fc3_W) + fc3_b
    
    # Layer 5 Activation.
    #fc3    = tf.nn.relu(fc3)
    
    # Layer 5 Drop-out
    #fc3 = tf.nn.dropout(fc3, keep_prob)
    
    # Layer 5: Fully Connected. Input = FC3_OUTPUT_SIZE. Output = n_classes.
    #fc4_W  = tf.Variable(tf.truncated_normal(shape=(FC3_OUTPUT_SIZE, n_classes), mean = mu, stddev = sigma))
    #fc4_b  = tf.Variable(tf.zeros(n_classes))
    #logits = tf.matmul(fc3, fc4_W) + fc4_b
    
    return logits, conv1_relu, conv2_relu

### Features and Labels

In [ ]:
x = tf.placeholder(tf.float32, (None, 32, 32, 1))
y = tf.placeholder(tf.int32, (None))
keep_prob = tf.placeholder(tf.float32)
one_hot_y = tf.one_hot(y, 43)

### Training Pipeline

**DPR**: As explained before, I experimented with a exponential decay applied to the learning rate... A small decay factor or 0.99 seemed to work best. Actually, I haven't observed a real improvement with respect of using a fixed learning rate, so I think this doesn't make much sense when using the TF AdamOptimizer.  It may  have made much more sense if using a regular Gradient Descent optimizer, but that is much slower.

In [ ]:
logits, conv1_relu, conv2_relu = MyLeNet(x, keep_prob)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits= logits, labels= one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)

##inserted
batch = tf.Variable(0, trainable=False)
starter_learning_rate = RATE
learning_rate = tf.train.exponential_decay(starter_learning_rate, batch * BATCH_SIZE, 34799, 0.99, staircase=True)
##end_insertado

#optimizer = tf.train.AdamOptimizer(learning_rate = RATE)
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
#training_operation = optimizer.minimize(loss_operation)
training_operation = optimizer.minimize(loss_operation, global_step = batch)


## Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

### Model Evaluation

In [ ]:
actual_prediction = tf.argmax(logits, 1)
correct_prediction = tf.equal(actual_prediction, tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

### Train the Model

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        print("EPOCH {} ...".format(i+1))
        lr = sess.run(learning_rate)
        print("Learning Rate: ", lr)
        
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: KEEP_DROPOUT})
            
        training_accuracy = evaluate(X_train, y_train)
        validation_accuracy = evaluate(X_valid, y_valid)

        print("Training Accuracy   = {:.3f}".format(training_accuracy))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './mylenet')
    print("Model saved")

**DPR** Not a bad result taking into account that the original LeNet-5 model of the lab class offered a validation accuracy of only 0.88, and 0.98 is much more that the required 0.93 to pass the project.

### Evaluate Results in Validation per every class.

In [ ]:
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, './mylenet')
    valid_images_pred = sess.run(actual_prediction,feed_dict={x:X_valid, keep_prob:1.0})

Let's analyze this data:

In [ ]:
print(len(valid_images_pred))
print(len(y_valid))

matches_correct = np.equal(valid_images_pred, y_valid)
stats_y = np.zeros_like(y_valid)

for i in range(len(matches_correct)):
    if matches_correct[i] == True:
        stats_y[ y_valid[i] ] += 1

for i in range(n_classes):
    total_per_class =(y_valid == i).sum()
    print("class {0:3d}   n_matches: {1:3d}   Total_per_class: {2:3d}   Ratio:{3:1.2f}"\
          .format(i, stats_y[i], total_per_class, stats_y[i]/total_per_class))


**DPR**: The List above shows the validation results per class (how many validation images per class correctly classified. The model behaves very well across all the classes.  Only classes 0, 27 and 41 show a low accuracy of 0.90. Not coincidently, these classes had very few samples in the validation and also in the original training set. Augmentation mitigates greatly this situation (the accuracy ratios for those classes were much lower without augmentation).

### Test the Model with test.p dataset file (as provided)

In [ ]:
import tensorflow as tf

saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, './mylenet')
    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

**DPR:** As expected, the Test Accuracy is lower that the Validation Accuracy, but still is pretty good.

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

import os
import numpy as np
import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from scipy.misc import imresize
get_ipython().magic('matplotlib inline')

list_images = os.listdir("new_images/")
#We exclude the images that begins with output
list_images = [file for file in list_images if not file.startswith('output') and not file.startswith('.')]

test_images = np.uint8(np.zeros((len(list_images),32,32,3)))

for i in range(len(list_images)):
    image_name = 'new_images/'+ list_images[i]
    #image = mpimg.imread(image_name)
    image = cv2.imread(image_name, cv2.IMREAD_COLOR)
    print('Loading image', image_name, 'of type', type(image), 'with dimensions:', image.shape)
    #image = imresize(image,(32,32), 'bicubic')
    
    b,g,r = cv2.split(image)
    test_images[i,:,:,:] = np.asarray(cv2.merge([r,g,b]), dtype='uint8')
    plt.figure()
    plt.imshow(test_images[i])


**DPR** I have obtained these images from Google Street View, mainly from the city of Cologne in Germany, so I think they can be considered like "real life" images. Some of them are quite distorted, or have distracting backgrounds, although in general are quite well illuminated 

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

import csv
import tensorflow as tf

#read the traffic signs classes names
with open('signnames.csv') as f:
  reader = csv.reader(f)
  signnames = list(reader)

test_norm_images = np.zeros((len(list_images),32,32))

#normalization (same as done before for the Training, Validation and Test Sets)
for i in range(len(test_images)):
    img = (.299*test_images[i,:,:,0] + .587*test_images[i,:,:,1] + .114*test_images[i,:,:,2])
    img = cv2.equalizeHist(img.astype(np.uint8))
    img = img.astype('float64')
    img -= np.mean(img)
    img /= np.std(img)
    test_norm_images[i,:,:] = img

# Reshape for tensorflow
test_norm_images = test_norm_images.reshape(test_norm_images.shape+(1,))

#load the saved session and perform prediction on the latest test images
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, './mylenet')
    test_images_pred = sess.run(actual_prediction, feed_dict={x: test_norm_images, keep_prob:1.0})

print("Predictions obtained. Please continue executing next cells.")

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

correct_classes_test = [40, 28, 35, 18, 4]
correct_labels_test = []
predicted_labels_test = []

for id in correct_classes_test:
    correct_labels_test.append((signnames[id+1][1]))
for id in test_images_pred:
    predicted_labels_test.append((signnames[id+1][1]))

i_image=0
for pred,correct in zip(predicted_labels_test, correct_labels_test):
    print("Image {0:1d} - Predicted Signal:{1:20s},  Actual Signal:{2:20s}".format(i_image,pred,correct))
    i_image+=1

print("Accuracy of the 5 New Images set: ", \
      (test_images_pred == correct_classes_test).sum()/len(test_images_pred))


**DPR**: There is some variability depending on the Training execution, which is a semi-random process. Usually Images 0 to 3 are always correctly classified, and Image 4 is usually mistaken because of the similarity of numbers 20 and 70.

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tk.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

#predict the top 5 labels for each test image
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, './mylenet')
    top_5_pred = sess.run(tf.nn.top_k(tf.nn.softmax(logits), 5), feed_dict={x: test_norm_images, keep_prob: 1.0})

In [ ]:
print(type(top_5_pred))

plt.figure(figsize=(16, 21))
for i in range(5):
    plt.subplot(5, 2, 2*i+1)
    plt.imshow(test_images[i], cmap='gray') 
    plt.title(i)
    plt.axis('off')
    plt.subplot(5, 2, 2*i+2)
    plt.barh(np.arange(1, 6, 1), top_5_pred.values[i, :])
    labs=[signnames[j+1][1] for j in top_5_pred.indices[i]]
    plt.yticks(np.arange(1, 6, 1), labs)
plt.show()

**DPR**: We see that the classifier is VERY sure of the first 4 images, but has serious doubts with the 70Km/h speed limit signal. The correct answer shows 3rd in the softmax probs.  A better model or the same model tuned for number recognition would probably improve these results. Also using colour instead of only Y illumination component would probably help (though not with this model.)

---

## Step 4: Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf TENSOR name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, index, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it maybe having trouble accessing the variable from inside a function

    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    #activation = sess.run(tf_activation, feed_dict={x : image_input, keep_prob:1.0})
    
    #print(activation.shape)
    featuremaps = activation.shape[3]
    
    #print(activation[0,:,:, 10])
    
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[index,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[index,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[index,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[index,:,:, featuremap], interpolation="nearest", cmap="gray")


#for v in tf.trainable_variables():
#    print(v.name)

saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, './mylenet')
    
    ### Note: When conv1_relu and conv2_rely only existed inside the MyLeNet function namespace, they could 
    ### still be accessed (apparently) using the get_tensor_by_name function, and everything executed without an error.
    ### HOWEVER, the tensor output was everything zeros, so all the plots were black.
    ### I was not been able to solve this issue, so now conv1_relu and conv2_relu are return parameters of the model.
    ### I could do simply:
    #   outputFeatureMap(test_norm_images, 3, conv1_relu)
    
    layer = tf.get_default_graph().get_tensor_by_name("conv1_relu:0")
    
    for i in range(len(test_norm_images)):
        outputFeatureMap(test_norm_images, i, layer, -1,-1, i)




**DPR - Note**: When conv1_relu and conv2_rely only existed inside the MyLeNet function namespace, they could 
still be accessed (apparently) using the get_tensor_by_name function, and everything executed without an error.
HOWEVER, the tensor output was everything zeros, so all the plots were black.
I was not been able to solve this issue, so now conv1_relu and conv2_relu are return parameters of the model.
I could do simply:
`outputFeatureMap(test_norm_images, 3, conv1_relu)`

### Question 9

Discuss how you used the visual output of your trained network's feature maps to show that it had learned to look for interesting characteristics in traffic sign images


**DPR:** It is very interesting how the first CONV layer output extracts the feature maps of the 5 test signals. And more interestingly even for me, is how each feature number across the 5 test signals seems to focus in some specific common feature. For example Feature 0 seems to focus on "left borders" while Feature 8 seems to focus on "top-right" borders.

This also demonstrates that the MyLeNet network learned to detect useful traffic sign features on its own. We never explicitly trained it to detect the signal's round or triangular shapes, numbers, figures, etc.

**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 